This notebook is started with the intention of looking at time dependance. It was created after the new hdf5 datascructure was created.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd

sys.path.insert(0, '../../drlib')
import drlib as dr

%matplotlib widget

In [ ]:
antData     = np.load('run1_antData_avgAll11776Spectra_gitignore.npy')
termData    = np.load('run1_termData_avgAll11776Spectra_gitignore.npy')
diffData    = dr.fft2Watts(antData) - dr.fft2Watts(termData)

plt.close('all')
plt.figure()
plt.plot(diffData[1:])

In [ ]:
sigTimeArr = np.zeros(11776)
vetoTimeArr = np.zeros(11776)

database = pd.read_hdf('/drBiggerBoy/drData/run1Data/preProcDataSet.hdf5', 'database_DF')
f = h5py.File('/drBiggerBoy/drData/run1Data/preProcDataSet.hdf5', 'r')

for i in range(11776):
    sigTimeArr[i] = f[str(i)]['diffSpec_W'][5368711]
    vetoTimeArr[i] = f[str(i)]['rigolSpec_W'][6400]

print(sigTimeArr.mean())


In [ ]:
plt.close('all')
plt.figure()
plt.plot(database.index, sigTimeArr)
plt.plot(database.index, vetoTimeArr*100)

In [ ]:
plt.close('all')
plt.figure()
plt.plot(database.index)